# Fatoração de Matrizes para Sistemas de Classificação de Machine Learning
## CCM0218 2020.2
### Fernando Valls Yoshida (11246714) e Lucas de Sousa Rosa (11296717)

In [3]:
import numpy as np
import math

## Rotação de Givens

In [4]:
def rot_Givens(W, n, m, i, j, c, s):
    # contando igual gente
    i = i - 1
    j = j - 1
    
    for r in range(m): # 0 até m - 1
        aux = c*W[i,r] - s*W[j,r]
        W[j,r] = s*W[i,r] + c*W[j,r]
        W[i,r] = aux

In [5]:
def cos_sin(W, i, j, k):
    i = i - 1
    j = j - 1
    k = k - 1
    
    if abs(W[i,k]) > abs(W[j,k]):
        t = -W[j,k]/W[i,k]
        c = 1/math.sqrt(1 + t**2)
        s = c*t
    else:
        t = -W[i, k]/W[j, k]
        s = 1/math.sqrt(1 + t**2)
        c = s*t    
    return c, s

## Exemplos para teste

In [6]:
W = np.array([[1.0,2.0], 
              [3.0,4.0]])
c, s = cos_sin(W, 1, 2, 1)

print("sin: {} \ncos: {}".format(s,c))

rot_Givens(W, 2, 2, 1, 2, c, s)
print(W)

sin: 0.9486832980505138 
cos: -0.3162277660168379
[[-3.16227766e+00 -4.42718872e+00]
 [ 1.11022302e-16  6.32455532e-01]]


In [7]:
A = np.array([[2.0, 1.0, 1.0,-1.0, 1.0],
              [0.0, 3.0, 0.0, 1.0, 2.0],
              [0.0, 0.0, 2.0, 2.0,-1.0],
              [0.0, 0.0,-1.0, 1.0, 2.0],
              [0.0, 0.0, 0.0, 3.0, 1.0]])

cos, sin = cos_sin(A, 3, 4, 3)

rot_Givens(A, 5, 5, 3, 4, cos, sin)
print(A)

[[ 2.          1.          1.         -1.          1.        ]
 [ 0.          3.          0.          1.          2.        ]
 [ 0.          0.          2.23606798  1.34164079 -1.78885438]
 [ 0.          0.          0.          1.78885438  1.34164079]
 [ 0.          0.          0.          3.          1.        ]]


## Resolução dos sistemas sobredeterminados

In [8]:
eps = 10e-10
def solve(W, b, n, m):
    for k in range(1, m + 1):
        for j in range(n, k, -1):
            i = j - 1
            if abs(W[j - 1, k - 1]) > eps:
                cos, sin = cos_sin(W, i, j, k)
                rot_Givens(W, n, m, i, j, cos, sin)
                rot_Givens(b, n, 1, i, j, cos, sin)
    
    #print(W)
    for k in range(m, 0, -1):
        soma = 0
        for j in range(k + 1, m + 1):
            soma += W[k - 1, j - 1]*b[j - 1, 0]
        b[k - 1, 0] = (b[k - 1, 0] - soma)/W[k - 1, k - 1]

## Exemplo

In [9]:
W = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = np.array([[1.0], [1.0]])

solve(W, b, 2, 2)

print("final \n{}".format(b))

final 
[[-1.]
 [ 1.]]


### Vários sistemas simultâneos

In [10]:
eps = 10e-10
def solve_multi(W, A, n, m, p):
    for k in range(1, p + 1):
        for j in range(n, k, -1):
            i = j - 1
            if abs(W[j - 1, k - 1]) > eps:
                cos, sin = cos_sin(W, i, j, k)
                rot_Givens(W, n, p, i, j, cos, sin)
                rot_Givens(A, n, m, i, j, cos, sin)
    
    #print(W)
    for k in range(p, 0, -1):
        for j in range(1, m + 1):
            soma = 0
            for i in range(k+1,p+1):
                soma += W[k-1,i-1]*A[i-1,j-1]
            A[k-1,j-1] = (A[k-1,j-1] - soma)/W[k-1,k-1]

In [12]:
W = np.array([[1.0, 2.0], [2.0, 1.0]])
A = np.array([[1.0,2.0,3.0],[4.0,5.0,6.0]])

solve_multi(W, A, 2, 3, 2)
print("final\n{}".format(A))

final
[[ 2.33333333  2.66666667  3.        ]
 [-0.66666667 -0.33333333  0.        ]]


### Fatoração por matrizes não-negativas

In [15]:
def erro(A,W,H,n,m):
    E = 0
    prod = np.multiply(W,H)
    for i in range(1,n+1):
        for j in range(1,m+1):
            E += (A[i-1,j-1] - prod[i-1,j-1])**2
    return E

In [16]:
def norm(W,n,p):
    for j in range(1,p+1):
        s = 0
        for i in range(1,n+1):
            s += (W[i-1,j-1])**2
        s = math.sqrt(s)
        for i in range(1,n+1):
            W[i-1,j-1] = W[i-1,j-1]/s

In [18]:
def set_H(A,p,m):
    H = np.zeros(p,m)
    for i in range(1,p+1):
        for j in range(1,m+1):
            if A[i-1,j-1] > 0:
                H[i-1,j-1] = A[i-1,j-1]
            else:
                H[i-1,j-1] = 0
    return H

In [ ]:
eps = 10e-5
t = 0
it_max = 100
def NMF(A,n,m,p):
    W = np.random.rand(n,p)
    A_copia = np.copy(A)
    # convenção: iteração ímpar e1, iteração par e2
    e1 = 0
    e2 = 0
    while ((abs(e1 - e2) < eps and t > 2) or t < it_max):
        norm(W,n,p)                # normalização de W
        solve_multi(W, A, n, m, p)
        H = set_H(A,p,m)